In [4]:
import os, json
import sys
import pandas as pd
import requests
from io import StringIO
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pd.options.plotting.backend = 'plotly'

for p in sys.path:
    if '99 scripts' in p.lower() :
        sys.path.remove(p)

sys.path.append('..')

import src

MainProcess


In [6]:
# init my api obj
api = src.StatisticsIcelandAPI()

# this obj provides a typable and readable interface to getting the endpoint urls 
economyEndPoint = src.constants.ENDPOINTS.Efnahagur

# define what endpoints I want to request
request_endpoints = [
    economyEndPoint.Prices_and_consumption.Consumer_price_index.CPI.Consumer_price_index_and_changes__base_1988_100, # this is just a url endpoint
    economyEndPoint.Employment_and_labour_productivity.Employment.Number_of_employed_persons__jobs_and_hours_worked_by_economic_activity_and_quarters__1991_to_2024, # as is this
    getattr(economyEndPoint.National_accounts.Gross_domestic_product,'National_accounts_-_quarterly').Quarterly_GDP_1995_to_2024, # the map to typable didn't work well for this one
    economyEndPoint.National_accounts.Financial_accounts.Money_and_credit.Weighted_average_interest_rates_of_commercial_banks_1960_to_2016,

]

# pass to api
api.add_endpoints(request_endpoints)

# and request
response = api.request()

# retrieve
CPI = response['Consumer_price_index_and_changes__base_1988_100']
Employment = response['Number_of_employed_persons__jobs_and_hours_worked_by_economic_activity_and_quarters__1991_to_2024']
GDP = response['Quarterly_GDP_1995_to_2024']
Interest = response['Weighted_average_interest_rates_of_commercial_banks_1960_to_2016']

Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Efnahagur/visitolur/1_vnv/1_vnv/VIS01000.px
Status: 200.
Remaining: 8.0


Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Efnahagur/vinnumagnogframleidni/vinnumagn/THJ11001.px
Status: 200.
Remaining: 7.0


Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Efnahagur/thjodhagsreikningar/landsframl/2_landsframleidsla_arsfj/THJ01601.px
Status: 200.
Remaining: 6.0


Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Efnahagur/thjodhagsreikningar/fjarmalareikningar/peningamal/PEN01101.px
Status: 200.
Remaining: 5.0




In [73]:
api = src.StatisticsIcelandAPI()

request_endpoints = [
        src.constants.ENDPOINTS.Ibuar.Population.Overview.Quarterly_data.Births__deaths_and_migration_by_sex_and_citizenship__NUTS3_regions_and_quarters_2011_to_2024,
        src.constants.ENDPOINTS.Ibuar.Population.Overview.Quarterly_data.Population_by_municipality__sex__citizenship_and_quarters_2011_to_2024,
        src.constants.ENDPOINTS.Ibuar.Population.Overview.Overview.Population_by_sex_and_age_1841_to_2025
        
    ]

api.add_endpoints(request_endpoints)

response = api.request()

Flux = response['Births__deaths_and_migration_by_sex_and_citizenship__NUTS3_regions_and_quarters_2011_to_2024']
Population_by_municipality = response['Population_by_municipality__sex__citizenship_and_quarters_2011_to_2024']
Population = response['Population_by_sex_and_age_1841_to_2025']


Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Ibuar/mannfjoldi/1_yfirlit/arsfjordungstolur/MAN10002.px
Status: 200.
Remaining: 9.0


Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Ibuar/mannfjoldi/1_yfirlit/arsfjordungstolur/MAN10001.px
Status: 200.
Remaining: 8.0


Request made to endpoint: https://px.hagstofa.is/pxen/api/v1/en/Ibuar/mannfjoldi/1_yfirlit/yfirlit_mannfjolda/MAN00101.px
Status: 200.
Remaining: 7.0




In [ ]:
df = convert_to_df(Flux)
(
    df.loc[lambda s: s.Event == 'Net migration',['Quarter','Total Total']]
    .plot(x='Quarter',y='Total Total',title='Net migration in Iceland 2011-2024')
)



In [130]:
(
    df.loc[lambda s: (s.Event == 'Births')+(s.Event == 'Deaths'),['Quarter','Event','Total Total']]
    .plot(x='Quarter',y='Total Total',title='Births and Deaths in Iceland 2011-2024',color='Event')
)

In [ ]:


df = convert_to_df(Population_by_municipality)
(
    df.loc[lambda s: s.Municipality == 'Total',['Quarter','Municipality','Foreign citizens','Icelandic citizens']]
    .melt(id_vars=['Quarter','Municipality'])
    .assign(value = lambda s: s.value.astype(float))
    .plot(x = 'Quarter', y = 'value', color = 'variable')
)



In [121]:
df = convert_to_df(Population)
(
    df.melt(id_vars=['Sex','Age'],var_name='Year',value_name='Population')
    .query('Age != "Total"')
    .assign(Age=lambda s: s.Age.str.replace(r'\syears?','',regex=True).replace('Under 1','0').astype(int),
            Year=lambda s: s.Year.astype(int),
            Population=lambda s: s.Population.replace('..','0').astype(int)
            )
    .groupby(['Year','Sex'])
    .agg({'Population':'sum'}).reset_index()
    .plot(x = 'Year',y='Population',color = 'Sex')
)

In [122]:
frame = (
    df.melt(id_vars=['Sex','Age'],var_name='Year',value_name='Population')
    .query('Age != "Total" & Sex != \'Total\' & Sex != \'Non-binary/Other\'')
    .assign(Age=lambda s: s.Age.str.replace(r'\syears?','',regex=True).replace('Under 1','0').astype(int),
            Year=lambda s: s.Year.astype(int),
            Population=lambda s: s.Population.replace('..','0').astype(int)
            )
    .query('Year == 2024')    
)

# Pivot for plotting
df_male = frame[frame['Sex'] == 'Males'].copy()
df_female = frame[frame['Sex'] == 'Females'].copy()
df_male['Population'] *= -1  # Invert male values for pyramid shape

# Sort ages from youngest to oldest
age_categories = sorted(frame['Age'].unique())

fig = go.Figure()

fig.add_trace(go.Bar(
    y=df_male['Age'],
    x=df_male['Population'],
    orientation='h',
    name='Male',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    y=df_female['Age'],
    x=df_female['Population'],
    orientation='h',
    name='Female',
    marker_color='red'
))

fig.update_layout(
    title='Population Pyramid',
    xaxis=dict(title='Population'),
    yaxis=dict(title='Age', categoryorder='category ascending'),
    barmode='overlay',
    bargap=0.1,
    template='plotly_white'
)

fig.show()

In [118]:
frame

,Sex,Age,Year,Population
81364,Males,0,2024,2258
81365,Males,1,2024,2317
81366,Males,2,2024,2599
81367,Males,3,2024,2450
81368,Males,4,2024,2356
...,...,...,...,...
81580,Females,105,2024,0
81581,Females,106,2024,1
81582,Females,107,2024,0
81583,Females,108,2024,0


In [26]:
def convert_to_df(response:requests.Request)->pd.DataFrame:
    return pd.read_csv(
        StringIO(response.text.lstrip('ï»¿')),
        sep=',',
        encoding='utf-8'
    ) 

In [4]:
def charts_Interest(Interest:requests.Request,variation:str)->go.Figure:
    return (
        convert_to_df(Interest)
        .replace(r'^\.+$','0.0', regex=True)
        .astype(float)
        .plot(x = 'Year', y = variation, title = variation, kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)
        .update_yaxes(title_text = '')
    )


charts_Interest(Interest,'General savings deposits Nominal interest, % per year')

In [5]:

def charts_grossDomesticProduct(GDP:requests.Response,value_unit:str,Category)->go.Figure:
    return (
        convert_to_df(GDP)
        .set_index(['Value unit','Category'])
        .loc[pd.IndexSlice[value_unit, Category,:], :]#.T
        .reset_index().T
        .iloc[2:].reset_index(names = 'Quarter')
        .rename(columns = {0:Category})
        .assign(**{Category: lambda x: x[Category].astype(float)})
        .plot(x='Quarter', y=Category, title = f"{Category} in {value_unit}", kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)   
        .update_yaxes(title_text = '')
    )

charts_grossDomesticProduct(GDP,value_unit= 'Current prices', Category = '1. Private final consumption')

In [6]:
def charts_consumerPriceIndex(CPI:requests.Response,variation:str)->go.Figure:
    return (
        convert_to_df(CPI)
        .plot(x = 'Month', y = variation, title = f"{variation} in Iceland", kind='line', markers=True, template='plotly_white')
        .update_layout(title_x = 0.5)
        .update_yaxes(title_text = '')
    )
charts_consumerPriceIndex(CPI, variation = 'Consumer price index Index')

In [7]:

def charts_EmploymentBySector(Employment:requests.Response)->go.Figure:
    return (
        convert_to_df(Employment)
        .pipe(lambda s: s.set_index(s.columns[:3].tolist()))
        .loc[pd.IndexSlice['Jobs','Total employment'],]
        .drop('Total - All activities')
        .T
        .reset_index(names = 'time')
        .melt(id_vars=['time'], var_name ='sector', value_name='Number of Jobs')
        .plot(x = 'time', y = 'Number of Jobs', color = 'sector',kind = 'area')
        .update_layout(showlegend = False)
    )   

charts_EmploymentBySector(Employment)


C:\Users\andnolan\AppData\Local\Temp\3\ipykernel_7460\1626104421.py:3: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [ ]:
import re

def to_snake_case(name:str)->str:
    """Convert a string to snake_case."""
    name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name)  # Add underscore between lowercase and uppercase letters
    name = re.sub(r'[^a-zA-Z0-9]+', '_', name)  # non-alphanumeric characters to underscore
    return name.lower().strip('_')  # Convert to lowercase

def create_SQL_command(resp:requests.Response, tableName:str)->str:
    mapping = {'object':'TEXT','int64':'INTEGER','float64':'REAL','string':'TEXT','Float64':'REAL','Int64':'INTEGER','boolean':'BOOLEAN'}
    df = convert_to_df(resp)
    # df.columns = [to_snake_case(col) for col in df.columns]
    df = df.replace(r'^\.+$','0', regex=True) # for some reason nan values arrive as . or .. 
    df = df.convert_dtypes()   
    dtypes = df.dtypes.replace(mapping).to_dict()

    sql_command = 'c.execute(\'DROP TABLE IF EXISTS {tableName};\')\n'
    sql_command += 'c.execute(\n\'\'\'\nCREATE TABLE IF NOT EXISTS {tableName} ('
    for column, dtype in dtypes.items():
        # snake_case = to_snake_case(column)
        sql_command += f"\n\t\"{column}\" {dtype},"
    sql_command = sql_command.rstrip(', \n') + '\n);\'\'\')\n\n'
    return sql_command.format(tableName = tableName)

CPI
Employment
GDP
Interest
# print(
#     create_SQL_command(CPI, 'CPI')
# )
# print(
#     create_SQL_command(Interest, 'Interest')
# )
# print(
#     create_SQL_command(Employment, 'Employment')
# )
# print(
#     create_SQL_command(GDP, 'GDP')
# )

print(
    create_SQL_command(Flux, 'Flux')
)
print(
    create_SQL_command(Population_by_municipality, 'Population_by_municipality')
)
print(
    create_SQL_command(Population, 'Population')
)



c.execute('DROP TABLE IF EXISTS Flux;')
c.execute(
'''
CREATE TABLE IF NOT EXISTS Flux (
	"Quarter" TEXT,
	"Event" TEXT,
	"Total Total" INTEGER,
	"Total Males" INTEGER,
	"Total Females" INTEGER,
	"Total Icelandic citizens" INTEGER,
	"Total Foreign citizens" INTEGER,
	"Capital region Total" INTEGER,
	"Capital region Males" INTEGER,
	"Capital region Females" INTEGER,
	"Capital region Icelandic citizens" INTEGER,
	"Capital region Foreign citizens" INTEGER,
	"Other regions Total" INTEGER,
	"Other regions Males" INTEGER,
	"Other regions Females" INTEGER,
	"Other regions Icelandic citizens" INTEGER,
	"Other regions Foreign citizens" TEXT
);''')


c.execute('DROP TABLE IF EXISTS Population_by_municipality;')
c.execute(
'''
CREATE TABLE IF NOT EXISTS Population_by_municipality (
	"Quarter" TEXT,
	"Municipality" TEXT,
	"Total" INTEGER,
	"Males" INTEGER,
	"Females" INTEGER,
	"Non-binary/Other" TEXT,
	"Icelandic citizens" INTEGER,
	"Foreign citizens" TEXT
);''')


c.execute('DROP TABLE IF EXISTS 

: 

In [ ]:
convert_to_df(CPI).replace(r'^\.+$',None,regex=True).drop(columns = 'Month').astype(float)

,Consumer price index Index,"Consumer price index Monthly change, %","Consumer price index Annual change, %","Consumer price index Annualized rate, latest month, %","Consumer price index Annualized rate, 3 month, %","Consumer price index Annualized rate, 6 month, %",Consumer price index less housing cost Index,"Consumer price index less housing cost Monthly change, %","Consumer price index less housing cost Annual change, %","Consumer price index less housing cost Annualized rate, latest month, %","Consumer price index less housing cost Annualized rate, 3 month, %","Consumer price index less housing cost Annualized rate, 6 month, %"
0,100.0,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
1,103.4,3.40,NaN,49.4,NaN,NaN,103.8,3.80,NaN,56.4,NaN,NaN
2,107.0,3.48,NaN,50.8,NaN,NaN,106.5,2.60,NaN,36.1,NaN,NaN
3,109.3,2.15,NaN,29.1,42.7,NaN,108.7,2.07,NaN,27.8,39.6,NaN
4,110.0,0.64,NaN,8.0,28.1,NaN,109.6,0.83,NaN,10.4,24.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
438,634.7,0.09,4.8,1.1,0.6,2.4,509.8,-0.20,2.7,-2.3,-1.8,0.5
439,637.2,0.39,4.8,4.8,3.1,2.2,511.7,0.37,2.8,4.6,1.9,0.4
440,635.5,-0.27,4.6,-3.2,0.9,0.7,510.2,-0.29,3.0,-3.5,-0.5,-1.1
441,641.3,0.91,4.2,11.5,4.2,2.4,515.6,1.06,2.7,13.5,4.6,1.4
